# Introduction. Discounted cash flow (DCF) 
DCF is a method of valuation used to determine the value of an investment based on future cash flows
First, we will walkthrough the valution, and then make a function to automate the process for any stock


In [1]:
import yfinance as yf
#importing yfinance library for access to Yahoo Finance API to pull data for our calculations

In [2]:
goog = yf.Ticker("goog") # Pick a stock
outstandingshares = goog.info['sharesOutstanding']  # Needed for future calculation

In [3]:
required_rate = 0.07 # Minimum return an investor will accept for owning a company's stock as compensation for a given level of risk associated with holding the stock
perpetual_rate = 0.03 # Same as terminal growth rate. The expected future revenue of the company
cash_flow_growth_rate = 0.02 
years = [4, 3, 2, 1]
free_cash_flow = [30972000, 42843000, 67012000, 60010000] # Data from finance yahoo 2022-2019 reverse order in thousand of $
# We will later create a scraping algorithm to get company's cashflow

In [4]:
future_cash_flow = [] # Cashflow forecast
discount_factor = [] # Interest rate which determines the present value of future cash flows 
dcf = [] # Discountedfuturecashflow
terminal_value = free_cash_flow[-1] * (1+perpetual_rate)/(required_rate-perpetual_rate)
# Termival value is the estimated value of a business beyond the forecasting period

In [5]:
for year in years:
    cashflow = free_cash_flow[-1] * (1+cash_flow_growth_rate)**year
    future_cash_flow.append(cashflow)
    discount_factor.append((1+required_rate)**year)


In [6]:
for i in range(0, len(years)):
    dcf.append(future_cash_flow[i]/discount_factor[i])

In [7]:
discounted_terminal_value = terminal_value / (1+required_rate)**4
dcf.append(discounted_terminal_value)
current_value = sum(dcf)
intrinsic_value = current_value * 1000 / outstandingshares
round(intrinsic_value,2)

237.0

Curent value of the stock is 132$. Lets readjust our assumptions based of Morgan Stanley report 

In [8]:
#readjusted assumptions
revenue_growth = 0.03
wacc = 0.078
terminal_growth_rate = 0.025

In [9]:
# Stock name -- stock's ticker
# Future cash flow growth rate -- our assumption
# WACC (Weighted average cost of capital) -- Same as the discount rate. We will use wacc as the required rate 
def dcf_valuation(stock_name: str, future_cash_flow_growth_rate: float, terminal_growth_rate: float, wacc: float):
    stock = yf.Ticker(stock_name) 
    outstandingshares = stock.info['sharesOutstanding'] 
    free_cash_flow_list = [] 
    for i in range(len(stock.cashflow.loc['Free Cash Flow'])):    # a loop created to get the data into a list from pandas data frame                  
        free_cash_flow_list.append(stock.cashflow.loc['Free Cash Flow'][i])
    free_cash_flow_list.reverse() #from oldest to newest
    terminal_value = free_cash_flow_list[-1] * (1+terminal_growth_rate)/(wacc-terminal_growth_rate)
    years = [1, 2, 3, 4]
    future_cash_flow = []
    discount_factor = []
    dcf = [] 
    for year in years:
        cashflow = free_cash_flow_list[-1] * (1+future_cash_flow_growth_rate)**year
        future_cash_flow.append(cashflow)
        discount_factor.append((1+wacc)**year)
    for i in range(0, len(years)):
        dcf.append(future_cash_flow[i]/discount_factor[i])
    discounted_terminal_value = terminal_value / (1+wacc)**4
    dcf.append(discounted_terminal_value)
    current_value = sum(dcf)
    intrinsic_value = current_value / outstandingshares
    return f'{round(intrinsic_value,2)}$'
    
    

    


In [10]:
dcf_valuation('goog', 0.05, 0.025, 0.078)

'184.59$'